In [154]:
import sys
from openbb import obb
sys.path.append(r'C:\Users\avido\Documents\other code\AlgoTrading')
from assets.api_credentials import openbb_pat
obb.account.login(pat=openbb_pat, remember_me=True)

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import mplfinance as mpf

from dsp_utils import find_similar_vectors, standardize
from collections import Counter

sys.path.append(r"C:/Users/avido/Documents/other code/Signal_processing_and_timeseries_exploration")

In [ ]:
#get max amount of hourly data
hourly_df = obb.equity.price.historical(symbol="iau", provider="yfinance", start_date="1900-01-01", interval='1h', include_prepost=True).to_df()
hourly_df.head()

In [151]:
keys_to_consider = ['open', 'high', 'low', 'close', 'volume']
# get the data for each day
daily_ohlc_vectors = []
daily_ohlc_dfs = []
for date in hourly_df['date'].unique():
    # yesterdays_date = hourly_df['date'].values[np.argwhere(hourly_df['date'].values == date)[0][0] - 1]
    # yesterdays_close = hourly_df[hourly_df['date'] == yesterdays_date]['close'].values[-1]
    
    daily_ohlc_vectors.append(np.hstack([
        standardize(hourly_df[hourly_df['date'] == date][key].values) for key in keys_to_consider
    ]))
    daily_ohlc_dfs.append(hourly_df[hourly_df['date'] == date])
    
lengths = [len(x) for x in daily_ohlc_vectors]
counts = Counter(lengths)
common_len = counts.most_common(1)[0][0]

idxs_to_remove = []
for i in range(len(daily_ohlc_vectors)):
    if daily_ohlc_vectors[i].shape[0] != common_len:
        idxs_to_remove.append(i)

daily_ohlc_vectors = [daily_ohlc_vectors[i] for i in range(len(daily_ohlc_vectors)) if i not in idxs_to_remove]
daily_ohlc_dfs = [daily_ohlc_dfs[i] for i in range(len(daily_ohlc_dfs)) if i not in idxs_to_remove]

    
daily_ohlc_array = np.array(daily_ohlc_vectors)

sim_vecs = find_similar_vectors(daily_ohlc_array, num_match = 10, metric = 'euclidean')

In [ ]:
sim_vecs

In [ ]:
for vec_idx in sim_vecs[0]:
    mpf.plot(daily_ohlc_dfs[vec_idx], type='candle', style='charles', volume=True)

In [ ]:
# now plot the day after each of the most similar vectors

for vec_idx in sim_vecs[0]:
    mpf.plot(daily_ohlc_dfs[vec_idx + 1], type='candle', style='charles', volume=True)